In [1]:
import os

import numpy as np
import vtk
from vtk.util.numpy_support import numpy_to_vtk, vtk_to_numpy

import pyfocusr
from pyfocusr.vtk_functions import read_vtk_mesh

In [2]:
location_meshes = '../data/'
n_points = '15k'

if n_points == '5k':
    target_filename = 'target_mesh.vtk'
    source_filename = 'source_mesh.vtk'
elif n_points == '15k':
    target_filename = 'target_mesh_15k.vtk'
    source_filename = 'source_mesh_15k.vtk'
    
target_vtk_mesh = read_vtk_mesh(os.path.join(location_meshes, target_filename))
source_vtk_mesh = read_vtk_mesh(os.path.join(location_meshes, source_filename))

Best results yet with: 

```
reg = pyfocusr.Focusr(vtk_mesh_target=target_vtk_mesh, 
                      vtk_mesh_source=source_vtk_mesh,
                      n_spectral_features=3,
                      n_extra_spectral=3,
                      get_weighted_spectral_coords=True,
                      list_features_to_calc=['curvature'],
                      rigid_reg_max_iterations=100,
                      non_rigid_alpha=0.001,
                      non_rigid_beta=4,
                      non_rigid_n_eigens=100,
                      non_rigid_max_iterations=300,
                      rigid_before_non_rigid_reg=False,
                      projection_smooth_iterations=40,
                      graph_smoothing_iterations=300,
                      include_points_as_features=True,
                      norm_physical_and_spectral=True,
                      feature_weights=np.diag([1,1]),
                      n_coords_spectral_ordering=10000)
```

In [3]:
reg = pyfocusr.Focusr(vtk_mesh_target=target_vtk_mesh, 
                      vtk_mesh_source=source_vtk_mesh,
                      n_spectral_features=3,
                      n_extra_spectral=3,
                      get_weighted_spectral_coords=True,
                      list_features_to_calc=['curvature'],
                      rigid_reg_max_iterations=100,
                      non_rigid_alpha=0.001,
                      non_rigid_beta=4,
                      non_rigid_n_eigens=100,
                      non_rigid_max_iterations=300,
                      rigid_before_non_rigid_reg=False,
                      projection_smooth_iterations=40,
                      graph_smoothing_iterations=300,
                      include_points_as_features=True,
                      norm_physical_and_spectral=True,
                      feature_weights=np.diag([1,1]),
                      n_coords_spectral_ordering=10000,
                      n_coords_spectral_registration=1000)

Loaded Mesh 1
Beginning Eigen Decomposition
Computed spectrum 1
Loaded Mesh 2
Beginning Eigen Decomposition
Computed spectrum 2


### View mesh features

In [4]:
reg.graph_source.eig_vals

array([0.00071329, 0.00089591, 0.00144346, 0.00182542, 0.0020771 ,
       0.00245536])

### View un-ordered/corrected/weighted eigenvectors (spectral coordinates)

#### Source Eigenvector 1

In [21]:
eig_vec = 2

In [22]:
reg.graph_source.view_mesh_eig_vec(eig_vec=eig_vec)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

#### Target Eigenvector 1

In [23]:
reg.graph_target.view_mesh_eig_vec(eig_vec=eig_vec)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

# Notes about comparison: 
Depending on if we include features ('curvature') or not, we will get different results for these meshes. 

If we leave the list at `list_features_to_calc` as an empty list `[]` then: <br>
We note that the color gradient is the same (in the same direction) between the two meshes. However, the colours are flipped. This is because the direction of "positive" can be flipped from one mesh to another. This will (should) be corrected automatically before the registration begins. 

If we include `'curvature'` in `list_features_to_calc` with `['curvature']` then: <br>
We get eigenvectors that align in direction and orientation (at least for the first 3 (0, 1, 2). 

Note, the flipped nature of the spectral coordinates can/will be corrected in the algorithm (if it exists). However, this only works because these are both of the same leg (right - as evident by the large side of the trochlear groove) and will likely faily/break if a left and right leg were analyzed. Therefore, all legs should be flipped to be the same side to "fix" things. 

## Begin Registration
This will print out the registration parameters/results as it goes. <br>
- First it will print out the "pairs" of eigenvectors between the two meshes and which ones on the source were flipped to match the target eigenvectors. 
- Second it will print the rigid registration progress and ultimately the registration "results"/"parameters"
- Third it will print the same for the non-rigid registration progress. 


In [8]:
reg.align_maps()

Eigenvector Sorting Results

The matches for eigenvectors were as follows:
Target	|  Source
     0	|  0     
     1	|  1     
     2	|  2     
     3	|  -4    
     4	|  3     
     5	|  5     
*Negative source values means those eigenvectors were flipped*

Appending Extra Features to Spectral Coords
Number of features (including spectral) used for registartion: 8

Non-Rigid (Deformable) Registration Beginning

Iteration:1
ML:-15421.111; 	ML change (error): 15421.111; 	Sigma^2:     0.016; 	Sigma^2 change:     0.053
[                                                                        ]
Iteration:2
ML:-19666.339; 	ML change (error):  4245.227; 	Sigma^2:     0.006; 	Sigma^2 change:     0.010
[                                                                        ]
Iteration:3
ML:-22754.293; 	ML change (error):  3087.954; 	Sigma^2:     0.002; 	Sigma^2 change:     0.004
[                                                                        ]
Iteration:4
ML:-26125.094; 	ML change (err

Iteration:47
ML:-31365.773; 	ML change (error):     0.350; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[===========                                                             ]
Iteration:48
ML:-31366.351; 	ML change (error):     0.578; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[===========                                                             ]
Iteration:49
ML:-31367.444; 	ML change (error):     1.093; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[===========                                                             ]
Iteration:50
ML:-31369.223; 	ML change (error):     1.780; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[============                                                            ]
Iteration:51
ML:-31371.608; 	ML change (error):     2.384; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[============                                                            ]
Iteration:52
ML:-31376.248; 	ML change (error):     4.640; 	Sigma^2:     0.000; 	Sigma^2 c

Iteration:98
ML:-31396.040; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[=======================                                                 ]
Iteration:99
ML:-31396.040; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[=======================                                                 ]
Iteration:100
ML:-31396.040; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[========================                                                ]
Iteration:101
ML:-31396.041; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[========================                                                ]
Iteration:102
ML:-31396.041; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[========================                                                ]
Iteration:103
ML:-31396.041; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma

Iteration:145
ML:-31396.047; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[==================================                                      ]
Iteration:146
ML:-31396.047; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[===================================                                     ]
Iteration:147
ML:-31396.047; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[===================================                                     ]
Iteration:148
ML:-31396.047; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[===================================                                     ]
Iteration:149
ML:-31396.047; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[===================================                                     ]
Iteration:150
ML:-31396.047; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sig

Iteration:194
ML:-31396.048; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[==============================================                          ]
Iteration:195
ML:-31396.048; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[==============================================                          ]
Iteration:196
ML:-31396.048; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[===============================================                         ]
Iteration:197
ML:-31396.048; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[===============================================                         ]
Iteration:198
ML:-31396.048; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[===============================================                         ]
Iteration:199
ML:-31396.048; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sig

Iteration:242
ML:-31396.049; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[==========================================================              ]
Iteration:243
ML:-31396.049; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[==========================================================              ]
Iteration:244
ML:-31396.049; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[==========================================================              ]
Iteration:245
ML:-31396.049; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[==========================================================              ]
Iteration:246
ML:-31396.049; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[===========================================================             ]
Iteration:247
ML:-31396.049; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sig

Iteration:290
ML:-31396.049; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[=====================================================================   ]
Iteration:291
ML:-31396.049; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[=====================================================================   ]
Iteration:292
ML:-31396.049; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:    -0.000
[======================================================================  ]
Iteration:293
ML:-31396.049; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[======================================================================  ]
Iteration:294
ML:-31396.049; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[======================================================================  ]
Iteration:295
ML:-31396.049; 	ML change (error):     0.000; 	Sigma^2:     0.000; 	Sig

array([  289,   530,   289, ..., 14640, 14639, 14648])

## View the spectral coordinates

In [9]:
reg.view_aligned_spectral_coords(starting_spectral_coord=0, include_rigid_aligned=True, include_unaligned=True)

Viewer(point_set_colors=array([[0.8392157 , 0.        , 0.        ],
       [0.54901963, 0.23137255, 1.       …

In [10]:
reg.view_meshes_colored_by_spectral_correspondences()

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [11]:
from itkwidgets import view

In [12]:
view(point_sets=[reg.smoothed_target_coords, reg.source_projected_on_target])

Viewer(geometries=[], gradient_opacity=0.22, point_set_colors=array([[0.8392157 , 0.        , 0.        ],
   …

In [13]:
view(point_sets=[reg.source_projected_on_target])

Viewer(geometries=[], gradient_opacity=0.22, point_set_colors=array([[0.8392157, 0.       , 0.       ]], dtype…

In [14]:
len(np.unique(reg.corresponding_target_idx_for_each_source_pt))

10862

In [15]:
reg.get_initial_correspondences()

In [16]:
len(np.unique(reg.corresponding_target_idx_for_each_source_pt))

11919

In [17]:
reg.view_meshes_colored_by_spectral_correspondences()

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…